<a href="https://colab.research.google.com/github/Jaejuna/SparkML/blob/main/Spark_MLlib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 7.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=6c5d1f19ae7569c03adadca0cfd7eb11a691a96012d7736e11ca01259e89aa01
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
# 데이터 다운
!gdown https://drive.google.com/uc?id=1qYh2aGOBZfUF9547M0pxAe17JhR7yLxm
!unzip sf-airbnb-clean.parquet.zip

Downloading...
From: https://drive.google.com/uc?id=1qYh2aGOBZfUF9547M0pxAe17JhR7yLxm
To: /content/sf-airbnb-clean.parquet.zip
100% 142k/142k [00:00<00:00, 63.6MB/s]
Archive:  sf-airbnb-clean.parquet.zip
   creating: sf-airbnb-clean.parquet/
  inflating: __MACOSX/._sf-airbnb-clean.parquet  
  inflating: sf-airbnb-clean.parquet/_committed_4320459746949313749  
  inflating: __MACOSX/sf-airbnb-clean.parquet/.__committed_4320459746949313749  
 extracting: sf-airbnb-clean.parquet/_started_4320459746949313749  
  inflating: __MACOSX/sf-airbnb-clean.parquet/.__started_4320459746949313749  
  inflating: sf-airbnb-clean.parquet/part-00000-tid-4320459746949313749-5c3d407c-c844-4016-97ad-2edec446aa62-6688-1-c000.snappy.parquet  
  inflating: __MACOSX/sf-airbnb-clean.parquet/._part-00000-tid-4320459746949313749-5c3d407c-c844-4016-97ad-2edec446aa62-6688-1-c000.snappy.parquet  
 extracting: sf-airbnb-clean.parquet/_SUCCESS  
  inflating: __MACOSX/sf-airbnb-clean.parquet/.__SUCCESS  


In [3]:
# 데이터 수집 및 탐색색
import pyspark
from pyspark.sql import SparkSession

#Create SparkSession
spark = SparkSession.builder.master("local[1]").appName("SparkByExamples.com").getOrCreate()

filePath = "./sf-airbnb-clean.parquet"
airbnbDF = spark.read.parquet(filePath)
airbnbDF.select("neighbourhood_cleansed", "room_type", "bedrooms", "bathrooms", "number_of_reviews", "price").show(5)

+----------------------+---------------+--------+---------+-----------------+-----+
|neighbourhood_cleansed|      room_type|bedrooms|bathrooms|number_of_reviews|price|
+----------------------+---------------+--------+---------+-----------------+-----+
|      Western Addition|Entire home/apt|     1.0|      1.0|            180.0|170.0|
|        Bernal Heights|Entire home/apt|     2.0|      1.0|            111.0|235.0|
|        Haight Ashbury|   Private room|     1.0|      4.0|             17.0| 65.0|
|        Haight Ashbury|   Private room|     1.0|      4.0|              8.0| 65.0|
|      Western Addition|Entire home/apt|     2.0|      1.5|             27.0|785.0|
+----------------------+---------------+--------+---------+-----------------+-----+
only showing top 5 rows



In [4]:
# 훈련 데이터셋과 테스트 데이터셋 분리리
trainDF, testDF = airbnbDF.randomSplit([.8, .2], seed=32)
print(f"""There are {trainDF.count()} rows in the training set, and {testDF.count()} in the test set""")

There are 5752 rows in the training set, and 1394 in the test set


In [5]:
# 변환기 transformer 사용하여 학습할 기능들 준비비
from pyspark.ml.feature import VectorAssembler

vecAssembler = VectorAssembler(inputCols=["bedrooms"], outputCol="features")
vecTrainDF = vecAssembler.transform(trainDF)
vecTrainDF.select("bedrooms", "features", "price").show(10)

+--------+--------+-----+
|bedrooms|features|price|
+--------+--------+-----+
|     1.0|   [1.0]|200.0|
|     1.0|   [1.0]|130.0|
|     1.0|   [1.0]| 85.0|
|     1.0|   [1.0]| 95.0|
|     3.0|   [3.0]|250.0|
|     1.0|   [1.0]| 45.0|
|     1.0|   [1.0]|115.0|
|     1.0|   [1.0]| 70.0|
|     1.0|   [1.0]|105.0|
|     1.0|   [1.0]| 86.0|
+--------+--------+-----+
only showing top 10 rows



In [6]:
# Linear Regression 모델 생성 및 학습습
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol='features', labelCol='price')
lrModel = lr.fit(vecTrainDF)

In [7]:
# 예측 가격값 알아보기기
m = round(lrModel.coefficients[0], 2)
b = round(lrModel.intercept, 2)
print(f"""The formula for the linear regression line is price = {m}*bedrooms + {b}""")

The formula for the linear regression line is price = 121.14*bedrooms + 52.02


In [8]:
# pipeline으로 간편화 해보기 (transformer)
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vecAssembler, lr])
pipelineModel = pipeline.fit(trainDF)

In [9]:
# pipeline으로 간편화 해보기 (estimator)
predDF = pipelineModel.transform(testDF)
predDF.select("bedrooms", "features", "price", "prediction").show(10)

+--------+--------+-----+------------------+
|bedrooms|features|price|        prediction|
+--------+--------+-----+------------------+
|     1.0|   [1.0]|250.0| 173.1567995347388|
|     1.0|   [1.0]|250.0| 173.1567995347388|
|     1.0|   [1.0]|100.0| 173.1567995347388|
|     1.0|   [1.0]|159.0| 173.1567995347388|
|     2.0|   [2.0]|199.0|294.29848575905436|
|     1.0|   [1.0]|100.0| 173.1567995347388|
|     1.0|   [1.0]|190.0| 173.1567995347388|
|     1.0|   [1.0]| 95.0| 173.1567995347388|
|     3.0|   [3.0]|200.0|415.44017198336985|
|     1.0|   [1.0]| 64.0| 173.1567995347388|
+--------+--------+-----+------------------+
only showing top 10 rows



In [13]:
# 범주형 데이터 다루기 : one-hot encoding
from pyspark.ml.feature import OneHotEncoder, StringIndexer

# 변환할 변수 지정 및 변환할 변수 명 생성
categoricalCols = [field for (field, dataType) in trainDF.dtypes if dataType == "string"]
indexOutputCols = [x + "Index" for x in categoricalCols]
oheOutputCols = [x + "OHE" for x in categoricalCols]

stringIndexer = StringIndexer(inputCols=categoricalCols, outputCols=indexOutputCols, handleInvalid = "skip")
oheEncoder = OneHotEncoder(inputCols=indexOutputCols, outputCols=oheOutputCols)

# 연속형 변수 지정
numericCols = [field for (field, dataType) in trainDF.dtypes if  ((dataType == "double") & (field != "price"))]
assemblerInputs = oheOutputCols + numericCols
vecAssembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")

In [14]:
# RFormula 사용하기
from pyspark.ml.feature import RFormula

rFormula = RFormula(formula = "price ~ .", 
                    featuresCol="features",
                    labelCol="price",
                    handleInvalid="skip")

In [16]:
# 평가지표 RMSE
from pyspark.ml.evaluation import RegressionEvaluator
regressionEvaluator = RegressionEvaluator(
    predictionCol="prediction",
    labelCol="price",
    metricName='rmse'
)
rmse = regressionEvaluator.evaluate(predDF)
print(f"RMSE is {rmse:.1f}")

RMSE is 164.1


In [17]:
# 평가지표 R^2 
r2 = regressionEvaluator.setMetricName("r2").evaluate(predDF)
print(f"R2 is {r2}")

R2 is 0.27548956205105046


In [19]:
from pyspark.ml.regression import DecisionTreeRegressor

# 모델 객체 생성
dt = DecisionTreeRegressor(labelCol="price")

# 숫자 열만 필터링
numericCols = [field for (field, dataType) in trainDF.dtypes
               if ((dataType == "double") & (field != "price"))]

# 위에서 정의한 StingIndexer의 출력과 숫자 열 결합
assemblerInputs = indexOutputCols + numericCols
vecAssembler = VectorAssembler(inputCols = assemblerInputs, outputCol="features")

# 단계를 파이프라인으로 결합
stages = [stringIndexer, vecAssembler, dt]
pipeline = Pipeline(stages = stages)

# 및 훈련 -> error
dt.setMaxBins(40)
pipelineModel = pipeline.fit(trainDF)

In [20]:
# 규칙 프린트
dtModel = pipelineModel.stages[-1]
print(dtModel.toDebugString)

DecisionTreeRegressionModel: uid=DecisionTreeRegressor_9ccdac40b5e7, depth=5, numNodes=49, numFeatures=33
  If (feature 12 <= 2.5)
   If (feature 12 <= 1.5)
    If (feature 5 in {1.0,2.0})
     If (feature 3 in {5.0,6.0,8.0,9.0,10.0,12.0,14.0,16.0,18.0,19.0,20.0,23.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,35.0})
      If (feature 3 in {8.0,14.0,16.0,19.0,26.0,29.0,30.0,32.0})
       Predict: 79.80661577608143
      Else (feature 3 not in {8.0,14.0,16.0,19.0,26.0,29.0,30.0,32.0})
       Predict: 107.79655172413793
     Else (feature 3 not in {5.0,6.0,8.0,9.0,10.0,12.0,14.0,16.0,18.0,19.0,20.0,23.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,35.0})
      If (feature 8 <= 37.743764999999996)
       Predict: 311.94594594594594
      Else (feature 8 > 37.743764999999996)
       Predict: 136.87947598253274
    Else (feature 5 not in {1.0,2.0})
     If (feature 3 in {0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,10.0,12.0,13.0,14.0,15.0,16.0,18.0,19.0,20.0,22.0,23.0,24.0,25.0,26.0,28.0,29.0,30.0,31.0,33.0

In [21]:
# 변수 중요도 출력
import pandas as pd

featureImp = pd.DataFrame(
    list(zip(vecAssembler.getInputCols(), dtModel.featureImportances)),
    columns = ["feature", "importance"]
)
featureImp.sort_values(by = "importance", ascending=False)

,feature,importance
12,bedrooms,0.272822
1,cancellation_policyIndex,0.157866
2,instant_bookableIndex,0.131703
15,number_of_reviews,0.118573
4,property_typeIndex,0.094791
3,neighbourhood_cleansedIndex,0.064283
20,review_scores_communication,0.044116
9,longitude,0.039507
14,minimum_nights,0.028713
0,host_is_superhostIndex,0.018161


In [26]:
# Random Forest
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor(labelCol="price", maxBins=40, seed = 42)
pipeline = Pipeline(stages = [stringIndexer, vecAssembler, rf])
pipelineModel = pipeline.fit(trainDF)

rfModel = pipelineModel.stages[-1]
featureImp = pd.DataFrame(
    list(zip(vecAssembler.getInputCols(), rfModel.featureImportances)),
    columns = ["feature", "importance"]
)
featureImp.sort_values(by = "importance", ascending=False)

,feature,importance
12,bedrooms,0.148366
1,cancellation_policyIndex,0.146566
10,accommodates,0.143950
3,neighbourhood_cleansedIndex,0.117988
14,minimum_nights,0.055242
8,latitude,0.048732
11,bathrooms,0.048505
13,beds,0.048285
4,property_typeIndex,0.042115
15,number_of_reviews,0.038880


In [33]:
pipeline = Pipeline(stages = [stringIndexer, vecAssembler, rf])

In [34]:
from pyspark.ml.tuning import ParamGridBuilder
paramGrid = (ParamGridBuilder().addGrid(rf.maxDepth, [2, 4, 6]).addGrid(rf.numTrees, [10, 100]).build())

In [35]:
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")

In [36]:
from pyspark.ml.tuning import CrossValidator

cv = CrossValidator(estimator=pipeline, evaluator=evaluator, estimatorParamMaps=paramGrid, numFolds=3, seed=32)
cvModel = cv.fit(trainDF)

In [37]:
list(zip(cvModel.getEstimatorParamMaps(), cvModel.avgMetrics))

[({Param(parent='RandomForestRegressor_a809306dbaa8', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 2,
   Param(parent='RandomForestRegressor_a809306dbaa8', name='numTrees', doc='Number of trees to train (>= 1).'): 10},
  311.2081409583495),
 ({Param(parent='RandomForestRegressor_a809306dbaa8', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 2,
   Param(parent='RandomForestRegressor_a809306dbaa8', name='numTrees', doc='Number of trees to train (>= 1).'): 100},
  310.07157754152513),
 ({Param(parent='RandomForestRegressor_a809306dbaa8', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 4,
   Param(parent='RandomForestRegressor_a809306dbaa8', nam

In [38]:
spark.stop()